# Coder House Data Engineering

Desarrollar un script que extraiga datos de una API pública. A su vez, el alumno debe crear una tabla en
Redshift para posterior carga de los datos extraidos.

In [1]:
import time
import psycopg2
import requests
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm

## Elección de la API

In [2]:
def obtener_datos_desde_api(codigo_pais, codigo_indicador):
    """
    Realiza una petición a la API del Banco Mundial para obtener datos de un indicador específico de un país.
    
    Parámetros:
    - codigo_pais (str): El código ISO del país.
    - codigo_indicador (str): El código del indicador económico.
    
    Retorna:
    - list: Lista de datos obtenidos de la API, o una lista vacía si hubo un error o no se encontraron datos.
    """
    url = f"https://api.worldbank.org/v2/country/{codigo_pais}/indicator/{codigo_indicador}?format=json&date=2000:2021"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    try:
        response = requests.get(url, headers = headers)
        if response.status_code == 200:
            data = response.json()
            if len(data) == 2 and 'page' in data[0]:
                return data[1]  # data[1] contiene los datos reales
    except requests.RequestException as Error:
        print(f"Error al obtener datos desde la API: {Error}")
    return []

def transformar_datos(datos_crudos):
    """
    Transforma los datos crudos obtenidos de la API en una lista de diccionarios, preparados para ser convertidos en DataFrame.
    
    Parámetros:
    - datos_crudos (list): Lista de datos crudos obtenidos de la API.
    
    Retorna:
    - list: Lista de diccionarios con los datos transformados, adecuados para conversión a DataFrame.
    """
    if datos_crudos:
        return [
            {'Country Code': dato.get('country', {}).get('id', 'N/A'),
             'Country Name': dato.get('country', {}).get('value', 'N/A'),
             'Indicator Code': dato.get('indicator', {}).get('id', 'N/A'),
             'Year': int(dato.get('date', '1900')),
             'Value': dato.get('value', None),}
            for dato in datos_crudos if dato.get('value') is not None
        ]
    return []

# Inicializar sesión de requests
session = requests.Session()

# Indicadores y países
indicadores = ['FR.INR.RINR', 'PA.NUS.FCRF', 'NY.GDP.MKTP.CD', 'NY.GDP.PCAP.CD', 'SP.POP.TOTL', 'FP.CPI.TOTL', 'GC.DOD.TOTL.GD.ZS', 'NE.EXP.GNFS.CD', 'NE.IMP.GNFS.CD', 'FI.RES.TOTL.CD', 'DT.DOD.DECT.CD', 'GC.TAX.TOTL.GD.ZS', 'SL.UEM.TOTL.NE.ZS', 'SH.XPD.CHEX.GD.ZS', 'SE.XPD.TOTL.GD.ZS', 'EG.ELC.ACCS.ZS', 'EG.USE.PCAP.KG.OE', 'EN.ATM.CO2E.PC', 'IT.NET.USER.ZS', 'SP.DYN.CBRT.IN']
indicadores_sanitizados = [indicador.replace('.', '_') for indicador in indicadores]
paises = ['MEX']

resultados = []
for pais in tqdm(paises, desc="Procesando países"):
    for indicador in tqdm(indicadores, desc = f"Indicadores para {pais}"):
        datos_crudos = obtener_datos_desde_api(pais, indicador)
        resultados.extend(transformar_datos(datos_crudos))

# Crear DataFrame
df = pd.DataFrame(resultados)

# Reemplazar puntos por guiones bajos en los nombres de los indicadores en el DataFrame
df['Indicator Code'] = df['Indicator Code'].str.replace('.', '_')
df['Value'] = df['Value'].round(2)
print(df)

Procesando países: 100%|█████████████████████████████████████████████████████████████████| 1/1 [00:17<00:00, 17.05s/it]

    Country Code Country Name  Indicator Code  Year  Value
0             MX       Mexico     FR_INR_RINR  2021   0.70
1             MX       Mexico     FR_INR_RINR  2020   1.45
2             MX       Mexico     FR_INR_RINR  2019   4.00
3             MX       Mexico     FR_INR_RINR  2018   2.70
4             MX       Mexico     FR_INR_RINR  2017   0.72
..           ...          ...             ...   ...    ...
401           MX       Mexico  SP_DYN_CBRT_IN  2004  22.35
402           MX       Mexico  SP_DYN_CBRT_IN  2003  22.80
403           MX       Mexico  SP_DYN_CBRT_IN  2002  23.26
404           MX       Mexico  SP_DYN_CBRT_IN  2001  23.72
405           MX       Mexico  SP_DYN_CBRT_IN  2000  24.16

[406 rows x 5 columns]


## Análisis del DataFrame

In [3]:
# Contar los valores únicos en cada columna, incluyendo valores NaN
valores_unicos = df.nunique(dropna = False)
print("\nValores únicos por columna:\n")
print(valores_unicos)


Valores únicos por columna:

Country Code        1
Country Name        1
Indicator Code     20
Year               22
Value             385
dtype: int64


In [4]:
# Seleccionar columnas que no son numéricas
columnas_no_numericas = df.select_dtypes(exclude = [np.number])
# Contar valores por cada columna no numérica
conteos_por_columna = {}
for columna in columnas_no_numericas.columns:
    conteos_por_columna[columna] = df[columna].value_counts(dropna = False)  # dropna=False incluye NaN en el conteo
# Imprimir los conteos de valores para cada columna no numérica
print("\nConteos de valores por cada columna no numérica:\n")
for columna, conteo in conteos_por_columna.items():
    print(f"Conteos para {columna}:\n{conteo}\n")


Conteos de valores por cada columna no numérica:

Conteos para Country Code:
Country Code
MX    406
Name: count, dtype: int64

Conteos para Country Name:
Country Name
Mexico    406
Name: count, dtype: int64

Conteos para Indicator Code:
Indicator Code
FR_INR_RINR          22
NE_IMP_GNFS_CD       22
IT_NET_USER_ZS       22
EG_ELC_ACCS_ZS       22
SH_XPD_CHEX_GD_ZS    22
SL_UEM_TOTL_NE_ZS    22
PA_NUS_FCRF          22
FI_RES_TOTL_CD       22
DT_DOD_DECT_CD       22
NE_EXP_GNFS_CD       22
FP_CPI_TOTL          22
SP_POP_TOTL          22
NY_GDP_PCAP_CD       22
NY_GDP_MKTP_CD       22
SP_DYN_CBRT_IN       22
EN_ATM_CO2E_PC       21
SE_XPD_TOTL_GD_ZS    20
EG_USE_PCAP_KG_OE    16
GC_TAX_TOTL_GD_ZS    15
GC_DOD_TOTL_GD_ZS     4
Name: count, dtype: int64



In [5]:
# Contar los valores nulos en cada columna
valores_faltantes = df.isnull().sum()
print("\nValores faltantes por columna:\n")
print(valores_faltantes)


Valores faltantes por columna:

Country Code      0
Country Name      0
Indicator Code    0
Year              0
Value             0
dtype: int64


In [6]:
# Calcular la media de cada columna numérica en el DataFrame
medias = df.select_dtypes(include = [np.number]).mean()
# Reemplazar los valores nulos en las columnas numéricas con su respectiva media
df.fillna(medias, inplace = True)
# Imprimir las columnas con valores actualizados para verificar
print("\nValores actualizados en las columnas numéricas:\n")
print(df)


Valores actualizados en las columnas numéricas:

    Country Code Country Name  Indicator Code  Year  Value
0             MX       Mexico     FR_INR_RINR  2021   0.70
1             MX       Mexico     FR_INR_RINR  2020   1.45
2             MX       Mexico     FR_INR_RINR  2019   4.00
3             MX       Mexico     FR_INR_RINR  2018   2.70
4             MX       Mexico     FR_INR_RINR  2017   0.72
..           ...          ...             ...   ...    ...
401           MX       Mexico  SP_DYN_CBRT_IN  2004  22.35
402           MX       Mexico  SP_DYN_CBRT_IN  2003  22.80
403           MX       Mexico  SP_DYN_CBRT_IN  2002  23.26
404           MX       Mexico  SP_DYN_CBRT_IN  2001  23.72
405           MX       Mexico  SP_DYN_CBRT_IN  2000  24.16

[406 rows x 5 columns]


In [7]:
# Detectar duplicados en cada fila del DataFrame
duplicados_por_filas_antes = df.duplicated().sum()
print("\nNúmero total de filas duplicadas antes de eliminar:\n")
print(duplicados_por_filas_antes)


Número total de filas duplicadas antes de eliminar:

0


In [8]:
# Eliminar filas duplicadas, manteniendo la primera aparición
df.drop_duplicates(inplace = True)
# Contar y mostrar el número de filas duplicadas después de eliminarlas
duplicados_por_filas_despues = df.duplicated().sum()
print("\nNúmero total de filas duplicadas después de eliminar:\n")
print(duplicados_por_filas_despues)


Número total de filas duplicadas después de eliminar:

0


In [9]:
# Detectar duplicados en cada columna del DataFrame
duplicados_por_columna = {col: df[col].duplicated().sum() for col in df.columns}
print("\nDuplicados por columna:\n")
print(duplicados_por_columna)


Duplicados por columna:

{'Country Code': 405, 'Country Name': 405, 'Indicator Code': 386, 'Year': 384, 'Value': 21}


In [10]:
# Obtener estadísticas descriptivas del DataFrame
estadisticas_descriptivas = df.describe(include = 'all')
print("\nEstadísticas descriptivas:\n")
print(estadisticas_descriptivas)


Estadísticas descriptivas:

       Country Code Country Name Indicator Code         Year         Value
count           406          406            406   406.000000  4.060000e+02
unique            1            1             20          NaN           NaN
top              MX       Mexico    FR_INR_RINR          NaN           NaN
freq            406          406             22          NaN           NaN
mean            NaN          NaN            NaN  2010.465517  1.237979e+11
std             NaN          NaN            NaN     6.322411  2.748893e+11
min             NaN          NaN            NaN  2000.000000 -1.250000e+00
25%             NaN          NaN            NaN  2005.000000  5.817500e+00
50%             NaN          NaN            NaN  2010.500000  9.725000e+01
75%             NaN          NaN            NaN  2016.000000  9.327692e+10
max             NaN          NaN            NaN  2021.000000  1.364508e+12


## Estructura de la tabla

In [11]:
def cargar_datos_a_redshift(df):
    """
    Conecta a una base de datos de Redshift, crea una tabla si no existe, 
    inserta datos desde un DataFrame de pandas y cierra la conexión.
    
    Parámetros:
        df (pandas.DataFrame): DataFrame que contiene los datos a cargar.
    
    Retorna:
        None
    """
    # Datos de configuración para la conexión
    url = 'data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com'
    base_de_datos = 'data-engineer-database'
    archivo_usuario = 'C:/Users/jeshu/Videos/GitHub/CoderHouseDataEngineering/Code/user_redshift.txt'
    archivo_contraseña = 'C:/Users/jeshu/Videos/GitHub/CoderHouseDataEngineering/Code/password_redshift.txt'
    
    # Leer el usuario y la contraseña de los archivos
    with open(archivo_usuario, 'r') as f:
        usuario = f.read().strip()
    with open(archivo_contraseña, 'r') as f:
        contraseña = f.read().strip()

    # Establecer la conexión con la base de datos de Redshift
    conn = psycopg2.connect(
        host=url,
        dbname=base_de_datos,
        user=usuario,
        password=contraseña,
        port=5439
    )
    conn.autocommit = True
    cursor = conn.cursor()

    # Crear tabla si no existe
    create_table_query = """
    DROP TABLE IF EXISTS economic_data;
    CREATE TABLE economic_data (
        "Country Code" VARCHAR NOT NULL,
        "Country Name" VARCHAR NOT NULL,
        "Indicator Code" VARCHAR NOT NULL,
        "Year" INTEGER NOT NULL,
        "Value" FLOAT,
        PRIMARY KEY ("Country Code", "Indicator Code", "Year")
    );
    """
    cursor.execute(create_table_query)

    # Insertar datos desde el DataFrame
    insert_query = """
    INSERT INTO economic_data ("Country Code", "Country Name", "Indicator Code", "Year", "Value") VALUES (%s, %s, %s, %s, %s);
    """
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Cargando datos"):
        cursor.execute(insert_query, (row['Country Code'], row['Country Name'], row['Indicator Code'], row['Year'], row['Value']))

    # Cerrar la conexión
    cursor.close()
    conn.close()

cargar_datos_a_redshift(df)

Cargando datos: 100%|████████████████████████████████████████████████████████████████| 406/406 [01:32<00:00,  4.39it/s]


## Variedad de los datos

In [12]:
def obtener_datos_de_redshift():
    """
    Establece una conexión con una base de datos de Redshift, ejecuta una consulta para 
    recuperar todos los datos de la tabla 'economic_data', convierte los datos en un 
    DataFrame de pandas y cierra la conexión.

    Retorna:
        pandas.DataFrame: DataFrame que contiene los datos recuperados de la tabla.
    """
    # Datos de configuración para la conexión
    url = 'data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com'
    base_de_datos = 'data-engineer-database'
    archivo_usuario = 'C:/Users/jeshu/Videos/GitHub/CoderHouseDataEngineering/Code/user_redshift.txt'
    archivo_contraseña = 'C:/Users/jeshu/Videos/GitHub/CoderHouseDataEngineering/Code/password_redshift.txt'
    
    # Leer el usuario y la contraseña de los archivos
    with open(archivo_usuario, 'r') as f:
        usuario = f.read().strip()
    with open(archivo_contraseña, 'r') as f:
        contraseña = f.read().strip()

    # Establecer la conexión con la base de datos
    conn = psycopg2.connect(
        host=url,
        dbname=base_de_datos,
        user=usuario,
        password=contraseña,
        port=5439
    )
    cursor = conn.cursor()

    # Ejecutar la consulta
    query = "SELECT * FROM economic_data;"
    cursor.execute(query)

    # Recuperar todos los datos
    data = cursor.fetchall()

    # Convertir los datos en un DataFrame para facilitar su manejo
    df = pd.DataFrame(data, columns=[desc[0] for desc in cursor.description])

    # Cerrar la conexión
    cursor.close()
    conn.close()

    return df

df = obtener_datos_de_redshift()
print(df)

    country code country name  indicator code  year  value
0             MX       Mexico     FR_INR_RINR  2021   0.70
1             MX       Mexico     FR_INR_RINR  2020   1.45
2             MX       Mexico     FR_INR_RINR  2019   4.00
3             MX       Mexico     FR_INR_RINR  2018   2.70
4             MX       Mexico     FR_INR_RINR  2017   0.72
..           ...          ...             ...   ...    ...
401           MX       Mexico  SP_DYN_CBRT_IN  2004  22.35
402           MX       Mexico  SP_DYN_CBRT_IN  2003  22.80
403           MX       Mexico  SP_DYN_CBRT_IN  2002  23.26
404           MX       Mexico  SP_DYN_CBRT_IN  2001  23.72
405           MX       Mexico  SP_DYN_CBRT_IN  2000  24.16

[406 rows x 5 columns]
